In [1]:
import numpy as np
import pandas as pd
import os
import glob
import shutil
# from osgeo import gdal
# import struct
# import utm
import matplotlib.pyplot as plt

# import sys
# To see koger_general_functions
# sys.path.append('/Users/blaircostelloe/Dropbox/HerdHover_project/drone-tracking-master/functions')
# import koger_general_functions as kgf

In [5]:
# This is the observation name you are working on 
video_name = 'observation070'

# These have to be manually gotten from the videos or frame extractions
# The first real visible instance of veritcal drone movement in the video
# first_moving_frames = [0, 150, 782]
# The first detectable rotation of the drone after "GoHome" is activated
# There is a excel file
go_home_frames = [2634, 11084, 9354]
# the clip number (within a flight) that go_home_frames are pulled from.
go_home_clips = [3, 2, 2]




# This assumes there is an outer folder that contains one folder for each 
# observation which then contains all of the actual drone logs from that observation
# for example:

# outer_drone_log_folder{
#     observation001 {
#         flightlog_ob001-01.csv
#         flightlog_ob001-02.csv
#         flightlog_ob001-03.csv
#     }
#     observation004 {
#         flightlog_ob004-01.csv
#         flightlog_ob004-02.csv
#     }
# }

# This is the path to the outer folder
outer_drone_log_folder = '/home/golden/Dropbox/herdhover-data/flight_logs'
drone_logs = glob.glob(outer_drone_log_folder + '/' + video_name + '/' '*')
drone_logs.sort()

# This is where the raw video images are saved 
frame_folders_root = '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage' + '/' + video_name
# This is the place the map images will be stored
output_folder_root = '/home/golden/kenya_drones/processing/kenya-tracking/map-images' + '/' + video_name
output_folder_root = '/home/golden/Dropbox/temp-image-viewing' + '/' + video_name

server_output_folder = os.path.join("/home/golden/kenya_drones/map_test_params", video_name)

if not os.path.isdir(output_folder_root):
    os.mkdir(output_folder_root)
    
    

num_flights = len(drone_logs)
if len(go_home_frames) != num_flights:
    raise ValueError('make sure you have as many first moving frames as drone flights in the observation')

In [6]:
csv_file = os.path.join(frame_folders_root, os.path.basename(frame_folders_root) + '.csv')
try:
    observation_df = pd.read_csv(csv_file)
except IOError:
    print('.csv file missing from folder')

# Get first and last frame number for every video segment in each drone flight

first_last_folders = []

# current video name
curr_video_name = None

for row in range(observation_df.shape[0]):
    # sample the video at every other frame
    # csv records raw frame numbers (say 60fps)
    first_frame = int(observation_df.loc[row, 'first_frame'])
    last_frame = int(observation_df.loc[row, 'last_frame'])
    
    # If new video name add a new list to keep track of that videos info
    if observation_df.loc[row, 'video_name'].rpartition('_')[0] != curr_video_name:
        first_last_folders.append([])
        curr_video_name = observation_df.loc[row, 'video_name'].rpartition('_')[0]

    if last_frame == -1:
        frames = glob.glob(frame_folders_root + '/' + observation_df.loc[row, 'video_name'] + '/*.jpg')
        frames.sort(key=lambda file: int(file.split('_')[-1].split('.')[0]))
        last_frame = int(frames[-1].split('.')[-2].split('_')[-1])
        
    first_last_folders[-1].append({'first_frame': first_frame, 'last_frame': last_frame})

In [7]:
first_last_folders

[[{'first_frame': 0, 'last_frame': 0},
  {'first_frame': 3742, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 2626}],
 [{'first_frame': 9786, 'last_frame': 19630},
  {'first_frame': 0, 'last_frame': 19634},
  {'first_frame': 0, 'last_frame': 11074},
  {'first_frame': 0, 'last_frame': 0}],
 [{'first_frame': 8528, 'last_frame': 19628},
  {'first_frame': 0, 'last_frame': 19636},
  {'first_frame': 0, 'last_frame': 9336},
  {'first_frame': 0, 'last_frame': 0}]]

In [8]:
print(csv_file)
df = pd.read_csv(csv_file)
df

/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/observation070.csv


,Unnamed: 0,index,Unnamed: 0.1,video_name,first_frame,last_frame
0,0,10,10,APR03_2018_A_DJI_0279,0,0
1,1,8,8,APR03_2018_A_DJI_0280,3742,-1
2,2,0,0,APR03_2018_A_DJI_0281,0,-1
3,3,4,4,APR03_2018_A_DJI_0282,0,2626
4,4,7,7,APR03_2018_B_DJI_0012,9786,-1
5,5,2,2,APR03_2018_B_DJI_0013,0,-1
6,6,1,1,APR03_2018_B_DJI_0014,0,11074
7,7,9,9,APR03_2018_B_DJI_0015,0,0
8,8,3,3,APR03_2018_C_DJI_0283,8528,-1
9,9,5,5,APR03_2018_C_DJI_0284,0,-1


In [9]:


#  first_last_folders = []
#  video_ind = -1
#  for folder_ind, folder in enumerate(video_frame_folders):
#      files = glob.glob(folder + '/*.jpg')
#      files.sort(key=lambda file: int(file.split('_')[-1].split('.')[0]))

#      first_frame = int(files[0].split('_')[-1].split('.')[0])
#      last_frame = int(files[-1].split('_')[-1].split('.')[0])
#      if first_frame == 0:
#          first_last_folders[video_ind].append({'first_frame': first_frame, 'last_frame': last_frame})
#      else:
#          video_ind += 1
#          first_last_folders.append([])
#          first_last_folders[video_ind].append({'first_frame': first_frame, 'last_frame': last_frame})

In [10]:
first_last_folders

[[{'first_frame': 0, 'last_frame': 0},
  {'first_frame': 3742, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 2626}],
 [{'first_frame': 9786, 'last_frame': 19630},
  {'first_frame': 0, 'last_frame': 19634},
  {'first_frame': 0, 'last_frame': 11074},
  {'first_frame': 0, 'last_frame': 0}],
 [{'first_frame': 8528, 'last_frame': 19628},
  {'first_frame': 0, 'last_frame': 19636},
  {'first_frame': 0, 'last_frame': 9336},
  {'first_frame': 0, 'last_frame': 0}]]

In [11]:
## to manually split into flights if all were combined into one flight:
#first_last_folders.append([])
#first_last_folders[1] = first_last_folders[0][5:9]
#first_last_folders[0] = first_last_folders[0][0:5]
#first_last_folders

In [12]:
# Get the first and last frame number for drone flight (merging all video segments in drone flight together)

first_last_flights = []

for folder_list in first_last_folders:
    first_frame = folder_list[0]['first_frame']
    last_frame = folder_list[0]['last_frame']
    for fl_dict in folder_list[1:]:
            last_frame += fl_dict['last_frame']
        
    first_last_flights.append({'first_frame': first_frame, 'last_frame': last_frame})

In [13]:
first_last_flights

[{'first_frame': 0, 'last_frame': 41890},
 {'first_frame': 9786, 'last_frame': 50338},
 {'first_frame': 8528, 'last_frame': 48600}]

In [14]:
# first_last_flights[0]['first_frame'] = 23370

In [15]:
flight_logs = []
for file in drone_logs:
    flight_log = pd.read_csv(file)
    # Get rid of potential white space in column names
    flight_log.rename(columns=lambda x: x.strip(), inplace=True)
    flight_logs.append(flight_log)

In [16]:
first_last_folders

[[{'first_frame': 0, 'last_frame': 0},
  {'first_frame': 3742, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 19632},
  {'first_frame': 0, 'last_frame': 2626}],
 [{'first_frame': 9786, 'last_frame': 19630},
  {'first_frame': 0, 'last_frame': 19634},
  {'first_frame': 0, 'last_frame': 11074},
  {'first_frame': 0, 'last_frame': 0}],
 [{'first_frame': 8528, 'last_frame': 19628},
  {'first_frame': 0, 'last_frame': 19636},
  {'first_frame': 0, 'last_frame': 9336},
  {'first_frame': 0, 'last_frame': 0}]]

In [17]:
home_frames =[]
for ind, log in enumerate(flight_logs):
    # first adjust the frame number to account for all the frames in previous clips for that flight.
    clip = go_home_clips[ind]
    df = log
    home_frame = go_home_frames[ind]
    for num in range(clip):
        home_frame = home_frame + int(first_last_folders[ind][num]['last_frame'])
    home_frames.append(home_frame)
    # now go through the log and find the row where flycState == "GoHome" AND compass heading has changed >.2
    # since "GoHome" was activated. Call this going_home
    compass_ref = df[df['flycState'] == 'Go_Home'].iloc[0]['compass_heading(degrees)']
    going_home = df[(df['flycState'] == 'Go_Home') & (abs(df['compass_heading(degrees)'] - compass_ref) >= 0.2)].iloc[0]
    # now assign 'frame_num' with value in going_home = to home_frame.
    df.loc[:, 'frame_num'] = ((df.loc[:, 'time(millisecond)'] - going_home['time(millisecond)']) / 16.666).astype(int) + home_frames[ind]
    #df.loc[:, 'frame_num'] = ((df.loc[:, 'time(millisecond)'] - df.loc[first_move[log_num], 'time(millisecond)']) / 16.666).astype(int) + first_moving_frames[log_num]

In [18]:
for ind, log in enumerate(flight_logs):
    log['altitude(m)'] = log['altitude(feet)']/3.28

In [19]:
# # Find which drone log files have isVideo = 1. 
# # First extract all the names of the drone log csv files
# root = '/Volumes/Seagate Desktop/kenya_drone_project/flight_logs'
# log_files = [os.path.join(path, name) for path, subdirs, files in os.walk(root) for name in files]
# log_files = [s for s in log_files if ".csv" in s]

# # Go through logs and check if the isVideo column contains a 1. If so, add the log to the list of "working" logs
# isVideo_yes = []
# isVideo_no = []
# for log in log_files:
#     df = pd.read_csv(log)
#     values = df.isVideo.unique()
#     if 1 in values:
#         isVideo_yes.append(log)
#     else:
#         isVideo_no.append(log)

In [20]:
# # Here we are matching up the drone log entries with the corresponding frames from the videos. We assume that
# # the drone's internal clock remains accurate throughout the entire flight and that there are actually one frame
# # captured every 1/60 seconds.
# # (1 second / 60 frames * (1000 ms / 1s) = 16.6... ms/frames)

# # Most straightforward is to find when the camera turned on (indicated in drone logs with isVideo == 1) and
# # assign that to frame 0. But sometimes isVideo does not work. For these logs we will line up the logs with 
# # the first vertical movement of the drone in the video.

# video_on = []
# log_num = 0

# # Find the first entries in each drone flight log where there is movement recorded
# first_move = [np.min(np.nonzero(flight_logs[flight_ind].loc[:, 'ascent(feet)'])) for flight_ind in range(num_flights)]
# print('The first log entries that correspond to vertical movement', first_move)

# for log in drone_logs:
#         df = flight_logs[log_num]
#         # check if log is in the list of logs for which isVideo actually worked
#         if log in isVideo_yes:
#             print("isVideo worked for" + log)
#             # find the first entry in the log where isVideo = 1
#             video_on = np.min(np.nonzero(df.loc[:, 'isVideo']))  
#             print("The first log entry with video recording is", video_on)
#             # create a new column called frame_num, which is the estimated frame number corresponding to the 
#             # row in the drone log. Frame 0 should be the first frame where isVideo = 1
#             df.loc[:, 'frame_num'] = (((df.loc[:, 'time(millisecond)'] - df.loc[video_on, 'time(millisecond)'])/16.666).astype(int))
#         else: 
#             # if isVideo didn't work, we need to use another marker to line up the logs and the videos
#             print('isVideo did not work for' + log + '. Using first movement instead.')
#             # Here we align drone logs and video frames based on the first observed vertical drone movement. 
#             df.loc[:, 'frame_num'] = ((df.loc[:, 'time(millisecond)'] - df.loc[first_move[log_num], 'time(millisecond)']) / 16.666).astype(int) + first_moving_frames[log_num]
#         log_num = log_num + 1

In [21]:
# Go through every entry and record frames where drone has moved by a certain threshold from previous recording
# Use these frames to feed pix4d to create map of space and get ground truth drone movement

# Get dataframe row numbers and frame numbers of images to use for map construction

#############
drone_movement_threshold = 0.00005

# List of lists to store the frame numbers being used in the map
frame_list = [[] for _ in range(len(drone_logs))]
# Record the log row index that corresponds to each image in the map 
# Each flight has a different list
ind_list = [[] for _ in range(len(drone_logs))]
for flight_ind in range(len(drone_logs)):

    # index in flight log corresponding to first frame in recording
    first_ind = np.min(np.nonzero(flight_logs[flight_ind]['frame_num'] >= first_last_flights[flight_ind]['first_frame']))
    # index in flight log corresponding to last frame in recording (-1 want row before first row after observation)
    last_ind = np.min(np.nonzero(flight_logs[flight_ind]['frame_num'] > first_last_flights[flight_ind]['last_frame'])) - 1

    # record the entries in the flight log the correspond to frames in the part of the video
    # over the animals
    # record whether the observation is happening in new column of the flight log dataframe
    flight_logs[flight_ind] = flight_logs[flight_ind].assign(observing = False)
    flight_logs[flight_ind].loc[first_ind:last_ind, 'observing'] = True
    
    # For images used in pix4d map, save the actual image name, none if not used in map
    flight_logs[flight_ind] = flight_logs[flight_ind].assign(image_name = None)

    # Add a new column to record the frames that are used to create the pix4d map
    flight_logs[flight_ind] = flight_logs[flight_ind].assign(used_in_map = False)

    # log = flight_logs[flight_ind].loc[first_ind:last_ind]
    log = flight_logs[flight_ind]
    
    # Always include the first observation frame in the generated map
    ind_list[flight_ind].append(first_ind)
    frame_list[flight_ind].append(log.loc[first_ind, 'frame_num'])
    flight_logs[flight_ind].loc[first_ind, 'used_in_map'] = True

    # The coordinates of the last image added to the map
    last_coord = np.array([log.loc[first_ind, 'latitude'], log.loc[first_ind, 'longitude']])
    # The log row index of the last saved frame
    last_saved_row_ind = first_ind
    last_heading = log.loc[first_ind, 'compass_heading(degrees)']
    
    # Is the drone curently rotating
    rotating = False
    # Is the drone currently moving
    moving = False
    # How often to take frame when it is already rotating (degrees)
    ###############
    rotating_thresh = 30
    # How many degrees the drone must turn for the drones to be considered rotating (degrees)
    start_rotating_thresh = 1
    # Speed theshold for moving (mph)
    start_moving_thresh = 1.5
    
    # For reference
    # how many times a frame was added to the map to deal with rotation
    rotate_count = 0
    # How many times the drone was considered moving
    move_count = 0
    thresh_count = 0
    
    heading_last_row = log.loc[first_ind, 'compass_heading(degrees)']


    for ind, row in log.loc[log['observing'] == True].iterrows():
        save_frame = False
        cur_coord = np.array([row['latitude'], row['longitude']])
        # How much has the drone rotated since the last saved frame
        heading_dif = np.abs(row['compass_heading(degrees)'] - last_heading) 
        if not rotating:
            if heading_dif > start_rotating_thresh:
                rotating = True
                save_frame = True
                rotate_count += 1
        elif rotating:
            # The drone is still rotating
            if heading_dif > rotating_thresh:
                save_frame = True
                rotate_count += 1
            # The drone has stopped rotating
            elif row['compass_heading(degrees)'] - heading_last_row == 0:
                rotating = False
                save_frame = True
                rotate_count += 1
        # Save the heading in the last row to see when the drone has stopped rotating
        heading_last_row = row['compass_heading(degrees)']
        # Starting to move
        if not moving and row['speed(mph)'] > start_moving_thresh:
            save_frame = True
            moving = True
            move_count += 1
        # Movement stopped
        if moving and row['speed(mph)'] == 0:
            save_frame = True
            moving = False
            move_count += 1

        # How much has the drone moved since the last saved frame
        diff = np.sqrt(np.sum(np.square(cur_coord - last_coord)))
        if diff > drone_movement_threshold:
            save_frame = True
            thresh_count += 1

        if save_frame:
            frame_list[flight_ind].append(row['frame_num'])
            ind_list[flight_ind].append(ind)
            flight_logs[flight_ind].loc[ind, 'used_in_map'] = True
            last_coord = cur_coord
            last_heading = row['compass_heading(degrees)'] 


    # Add the last frame of the observation no matter what
    if not log.loc[last_ind, 'used_in_map']:
        ind_list[flight_ind].append(last_ind)
        frame_list[flight_ind].append(log.loc[last_ind, 'frame_num'])
        flight_logs[flight_ind].loc[last_ind, 'used_in_map'] = True


    print('{} frames were added to the map from flight {}.'.format(len(set(frame_list[flight_ind])), flight_ind))
    # print(list(flight_logs[flight_ind].loc[flight_logs[flight_ind]['used_in_map'], 'frame_num']))
    print('{} frames were added from rotation'.format(rotate_count))
    print('{} frames were added from movement'.format(move_count))
    print('{} frames were added from continued movement'.format(thresh_count))
    

print('{} frames were added to the map in total'.format(sum([len(set(l)) for l in frame_list])))


/home/golden/coding-py36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


79 frames were added to the map from flight 0.
19 frames were added from rotation
14 frames were added from movement
45 frames were added from continued movement
2 frames were added to the map from flight 1.
0 frames were added from rotation
0 frames were added from movement
0 frames were added from continued movement
149 frames were added to the map from flight 2.
65 frames were added from rotation
24 frames were added from movement
62 frames were added from continued movement
230 frames were added to the map in total


In [22]:
# This is for checking map accuracy
# Add 100 extra frames that will be used in the map to compare movement estimates vs reality

num_test_frames_per_log = 50

for flight_ind in range(len(drone_logs)):
    flight_logs[flight_ind] = flight_logs[flight_ind].assign(test_frame=False)
    # index in flight log corresponding to first frame in recording
    first_ind = np.min(np.nonzero(flight_logs[flight_ind]['frame_num'] >= first_last_flights[flight_ind]['first_frame']))
    # index in flight log corresponding to last frame in recording
    last_ind = np.min(np.nonzero(flight_logs[flight_ind]['frame_num'] >= first_last_flights[flight_ind]['last_frame']))
    for ind in np.linspace(first_ind, last_ind, num_test_frames_per_log+1, endpoint=False, dtype=int)[1:]:
        # first_ind is already in the map so don't use that
        flight_logs[flight_ind].loc[ind, 'used_in_map'] = True
        flight_logs[flight_ind].loc[ind, 'test_frame'] = True
        frame_list[flight_ind].append(flight_logs[flight_ind].loc[ind, 'frame_num'])
        ind_list[flight_ind].append(ind)

In [23]:
glob_string = frame_folders_root + '/*_*_*_*_*'
glob.glob(glob_string)

['/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_A_DJI_0281',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_B_DJI_0014',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_B_DJI_0013',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_C_DJI_0283',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_A_DJI_0282',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_C_DJI_0284',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_C_DJI_0286',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_B_DJI_0012',
 '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-foo

In [24]:
# print(flight_logs[0].loc[flight_logs[0]['used_in_map'], ['image_name', 'longitude', 'latitude', 'test_frame']])

[[6247, 23941, 24139, 24331, 24517, 24715, 24913, 27458, 33950, 34208, 48392, 48578, 55551, 62643, 62739, 67125], [], []]

In [25]:
# Get the actual frames chosen above to construct map

# Should the actual images be copied somewhere
SAVE_IMAGES = True

glob_string = frame_folders_root + '/*_*_*_*_*'
video_frame_folders = [fn for fn in glob.glob(glob_string) 
                       if not (os.path.basename(fn).startswith('drone') 
                       or os.path.basename(fn).startswith('observation'))]


video_names = [folder.rpartition('_')[0] for folder in video_frame_folders]
# have only one entry for every video name
video_names = list(set(video_names))
video_names.sort()
print(video_names)

# This is where the images will be saved
map_images_folder = os.path.join(output_folder_root, 'map-images')
if not os.path.isdir(map_images_folder):
    os.makedirs(map_images_folder, exist_ok=True)

video_ind = 0
export_images = []
for video_ind in range(len(video_names)):
    # Folders for all the images extracted from the clips of a given flight
    image_folders = glob.glob(video_names[video_ind] + '*')
    image_folders.sort()

    


    fl_dicts = first_last_folders[video_ind]
    # Frame_list contains the frame numbers that should be added to the map
    for map_frame_ind, frame_num in enumerate(frame_list[video_ind][:]):
       # ex_frame_num = 0
        for folder_ind, fl_dict in enumerate(fl_dicts):
            # Frame is contained the this folder
            if frame_num <= fl_dict['last_frame']:
                # make frame num even since those are the only frames that get extracted
                if frame_num % 2 != 0:
                    frame_num -= 1
                image_file = glob.glob(image_folders[folder_ind] + '/*_*' + '{:05d}'.format(frame_num) + '.jpg')
                export_images.append(image_file[0])
                if not os.path.exists(image_file[0]):
                    print('following file does not exist: {}'.format(image_file[0]))
                else:
                    out_file = os.path.join(map_images_folder, os.path.basename(image_file[0]))
                    flight_logs[video_ind].loc[ind_list[video_ind][map_frame_ind], 'image_name'] = os.path.basename(image_file[0])
                    if SAVE_IMAGES:
                        shutil.copyfile(image_file[0], out_file)

                break
            else:
#               frame number is in a later folder
                frame_num -= (fl_dict['last_frame'] + 2)
#               ex_frame_num += (fl_dict['last_frame'] + 2) 
                if folder_ind >= len(fl_dicts) - 1:
                    print('failed', frame_list[video_ind][map_frame_ind])



print('This is where the images will be saved:', map_images_folder)

['/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_A_DJI', '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_B_DJI', '/home/golden/kenya_drones/processing/kenya-tracking/raw-frames/raw-footage/observation070/APR03_2018_C_DJI']
This is where the images will be saved: /home/golden/Dropbox/temp-image-viewing/observation070/map-images


In [26]:
out_file

'/home/golden/Dropbox/temp-image-viewing/observation070/map-images/APR03_2018_C_DJI_0285_08558.jpg'

In [27]:
flight_num = 1
print(flight_logs[flight_num].loc[flight_logs[flight_num]['used_in_map']].shape[0])
print(len(frame_list[flight_num]))

52
52


In [28]:
# Save the dataframes in the output folder specified above
dropbox_folder = os.path.join("/home/golden/Dropbox/map-params", video_name, 'drone-logs')
if not os.path.isdir(dropbox_folder):
    os.makedirs(dropbox_folder)
for flight_num in range(len(flight_logs)):
    filename = os.path.join(dropbox_folder, 'flight_{}.pkl'.format(flight_num))
    flight_logs[flight_num].to_pickle(filename)

In [29]:
server_output_folder

'/home/golden/kenya_drones/map_test_params/observation070'

In [30]:
print(frame_list[1][0])
print(first_last_folders[1][0])

9787
{'first_frame': 9786, 'last_frame': 19630}


In [31]:
# Save the information about the frames used in map in .csv
# saved to output folder specified above
dfs = []
for flight_ind in range(len(flight_logs)):
    dfs.append(flight_logs[flight_ind].loc[ind_list[flight_ind], ['image_name', 'latitude', 'longitude', 'altitude(m)']].copy())
total_df = pd.concat(dfs, ignore_index=True)

total_df.to_csv(os.path.join(output_folder_root, 'whole_obs_file_lat_long_alt.csv'), header=False, index=False)

In [24]:
total_df.shape

(255, 4)

In [20]:
 for flight_ind in range(3):
     print(flight_logs[flight_ind].loc[flight_logs[flight_ind]['used_in_map'], ['image_name']])

                            image_name
1353   NOV14_2017_E_DJI_0231_08214.jpg
1401   NOV14_2017_E_DJI_0231_08502.jpg
1430   NOV14_2017_E_DJI_0231_08688.jpg
1565   NOV14_2017_E_DJI_0231_09498.jpg
1600   NOV14_2017_E_DJI_0231_09708.jpg
1610   NOV14_2017_E_DJI_0231_09774.jpg
1612   NOV14_2017_E_DJI_0231_09786.jpg
1620   NOV14_2017_E_DJI_0231_09834.jpg
1629   NOV14_2017_E_DJI_0231_09888.jpg
1643   NOV14_2017_E_DJI_0231_09972.jpg
1654   NOV14_2017_E_DJI_0231_10038.jpg
1663   NOV14_2017_E_DJI_0231_10092.jpg
1680   NOV14_2017_E_DJI_0231_10194.jpg
1693   NOV14_2017_E_DJI_0231_10272.jpg
1701   NOV14_2017_E_DJI_0231_10320.jpg
1705   NOV14_2017_E_DJI_0231_10344.jpg
1716   NOV14_2017_E_DJI_0231_10410.jpg
1720   NOV14_2017_E_DJI_0231_10434.jpg
1723   NOV14_2017_E_DJI_0231_10452.jpg
1725   NOV14_2017_E_DJI_0231_10464.jpg
1729   NOV14_2017_E_DJI_0231_10488.jpg
1736   NOV14_2017_E_DJI_0231_10530.jpg
1745   NOV14_2017_E_DJI_0231_10584.jpg
1747   NOV14_2017_E_DJI_0231_10596.jpg
3223   NOV14_2017_E_DJI_0

IndexError: list index out of range

In [21]:
flight_logs[2].iloc[10522]

latitude                               0.337653
longitude                               36.9062
altitude(feet)                          5662.69
ascent(feet)                             265.42
speed(mph)                                    0
distance(feet)                          2247.62
time(millisecond)                       1103500
datetime(utc)               2018-04-04 06:19:25
satellites                                   20
gpslevel                                      0
voltage(v)                               14.608
max_altitude(feet)                       5711.9
max_ascent(feet)                        314.633
max_speed(mph)                          28.7447
max_distance(feet)                      2247.72
xSpeed(mph)                                   0
ySpeed(mph)                                   0
zSpeed(mph)                                   0
compass_heading(degrees)                  261.9
pitch(degrees)                              1.1
roll(degrees)                           